In [1]:
# Cell 1: 패키지 설치 및 임포트
# !pip install langchain langchain-openai python-dotenv openai numpy

import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAIEmbeddings
import openai
import numpy as np

In [2]:
# Cell 2: Azure OpenAI 임베딩 설정 로드
load_dotenv()

# 임베딩 모델 환경 변수
embedding_endpoint = os.getenv("AZURE_OPENAI_EMBEDDING_ENDPOINT")
embedding_api_key = os.getenv("AZURE_OPENAI_EMBEDDING_API_KEY")
embedding_api_version = os.getenv("AZURE_OPENAI_EMBEDDING_API_VERSION")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")

# 설정 확인
print("✅ Azure OpenAI 임베딩 설정:")
print(f"Endpoint: {embedding_endpoint}")
print(f"Deployment: {embedding_deployment}")
print(f"Model: {embedding_model_name}")
print(f"API Version: {embedding_api_version}")
print(f"API Key: {'***' + embedding_api_key[-4:] if embedding_api_key else 'Not Found'}")

✅ Azure OpenAI 임베딩 설정:
Endpoint: https://hspar-m7k2pfor-swedencentral.openai.azure.com/
Deployment: text-embedding-ada-002
Model: text-embedding-ada-002
API Version: 2025-01-01-preview
API Key: ***RdJz


In [3]:
# Cell 3: LangChain Azure OpenAI 임베딩 초기화
try:
    embeddings = AzureOpenAIEmbeddings(
        azure_endpoint=embedding_endpoint,
        api_key=embedding_api_key,
        api_version=embedding_api_version,
        azure_deployment=embedding_deployment,
        model=embedding_model_name
    )
    
    print("✅ LangChain Azure OpenAI 임베딩 모델이 초기화되었습니다.")
    
except Exception as e:
    print(f"❌ LangChain 임베딩 초기화 실패: {e}")
    embeddings = None

✅ LangChain Azure OpenAI 임베딩 모델이 초기화되었습니다.


In [4]:
# Cell 4: 단일 텍스트 임베딩 테스트
def test_single_embedding():
    """
    단일 텍스트의 임베딩 생성 테스트
    """
    if embeddings is None:
        print("❌ 임베딩 모델이 초기화되지 않았습니다.")
        return False
    
    test_text = "안녕하세요! 이것은 임베딩 테스트용 텍스트입니다."
    
    try:
        print(f"테스트 텍스트: {test_text}")
        
        # 임베딩 생성
        embedding_vector = embeddings.embed_query(test_text)
        
        print(f"\n✅ 임베딩 생성 성공!")
        print(f"임베딩 차원: {len(embedding_vector)}")
        print(f"임베딩 타입: {type(embedding_vector)}")
        print(f"처음 10개 값: {embedding_vector[:10]}")
        print(f"임베딩 범위: {min(embedding_vector):.6f} ~ {max(embedding_vector):.6f}")
        
        return True
        
    except Exception as e:
        print(f"❌ 임베딩 생성 실패: {e}")
        return False

# 단일 임베딩 테스트
single_success = test_single_embedding()

테스트 텍스트: 안녕하세요! 이것은 임베딩 테스트용 텍스트입니다.

✅ 임베딩 생성 성공!
임베딩 차원: 1536
임베딩 타입: <class 'list'>
처음 10개 값: [-0.011808423325419426, -0.016904553398489952, -0.0037243454717099667, -0.026901308447122574, 0.00277452333830297, 0.023773249238729477, -0.01388076227158308, 0.006392971146851778, -0.019641604274511337, -0.00407951045781374]
임베딩 범위: -0.642712 ~ 0.239609


In [5]:
# Cell 5: 여러 텍스트 임베딩 테스트
def test_multiple_embeddings():
    """
    여러 텍스트의 임베딩 생성 및 유사도 계산
    """
    if embeddings is None:
        print("❌ 임베딩 모델이 초기화되지 않았습니다.")
        return False
    
    test_texts = [
        "머신러닝은 인공지능의 한 분야입니다.",
        "딥러닝은 머신러닝의 하위 분야입니다.",
        "오늘 날씨가 정말 좋네요.",
        "Python은 프로그래밍 언어입니다.",
        "AI와 ML은 밀접한 관련이 있습니다."
    ]
    
    try:
        print("테스트 텍스트들:")
        for i, text in enumerate(test_texts, 1):
            print(f"{i}. {text}")
        
        # 여러 텍스트 임베딩
        embedding_vectors = embeddings.embed_documents(test_texts)
        
        print(f"\n✅ 임베딩 생성 성공!")
        print(f"임베딩 개수: {len(embedding_vectors)}")
        print(f"각 임베딩 차원: {len(embedding_vectors[0])}")
        
        # 유사도 계산 (코사인 유사도)
        def cosine_similarity(vec1, vec2):
            return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
        
        print(f"\n📊 텍스트 간 유사도 (코사인 유사도):")
        for i in range(len(test_texts)):
            for j in range(i+1, len(test_texts)):
                similarity = cosine_similarity(embedding_vectors[i], embedding_vectors[j])
                print(f"텍스트 {i+1} ↔ 텍스트 {j+1}: {similarity:.4f}")
        
        return True
        
    except Exception as e:
        print(f"❌ 여러 텍스트 임베딩 실패: {e}")
        return False

# 여러 임베딩 테스트
multiple_success = test_multiple_embeddings()

테스트 텍스트들:
1. 머신러닝은 인공지능의 한 분야입니다.
2. 딥러닝은 머신러닝의 하위 분야입니다.
3. 오늘 날씨가 정말 좋네요.
4. Python은 프로그래밍 언어입니다.
5. AI와 ML은 밀접한 관련이 있습니다.

✅ 임베딩 생성 성공!
임베딩 개수: 5
각 임베딩 차원: 1536

📊 텍스트 간 유사도 (코사인 유사도):
텍스트 1 ↔ 텍스트 2: 0.9414
텍스트 1 ↔ 텍스트 3: 0.7530
텍스트 1 ↔ 텍스트 4: 0.8501
텍스트 1 ↔ 텍스트 5: 0.8808
텍스트 2 ↔ 텍스트 3: 0.7375
텍스트 2 ↔ 텍스트 4: 0.8405
텍스트 2 ↔ 텍스트 5: 0.8554
텍스트 3 ↔ 텍스트 4: 0.7535
텍스트 3 ↔ 텍스트 5: 0.7384
텍스트 4 ↔ 텍스트 5: 0.8123


In [6]:
# Cell 6: OpenAI 클라이언트로 직접 임베딩 테스트
def test_openai_client_embedding():
    """
    OpenAI 클라이언트로 직접 임베딩 생성
    """
    try:
        client = openai.AzureOpenAI(
            azure_endpoint=embedding_endpoint,
            api_key=embedding_api_key,
            api_version=embedding_api_version
        )
        
        test_text = "OpenAI 클라이언트로 임베딩을 생성합니다."
        
        response = client.embeddings.create(
            model=embedding_deployment,
            input=test_text
        )
        
        embedding_vector = response.data[0].embedding
        
        print("✅ OpenAI 클라이언트 임베딩 성공:")
        print(f"텍스트: {test_text}")
        print(f"임베딩 차원: {len(embedding_vector)}")
        print(f"임베딩 벡터 (처음 5개): {embedding_vector[:5]}")
        
        # 토큰 사용량 확인
        if hasattr(response, 'usage') and response.usage:
            print(f"\n📊 토큰 사용량:")
            print(f"  - 입력 토큰: {response.usage.prompt_tokens}")
            print(f"  - 총 토큰: {response.usage.total_tokens}")
        
        return True
        
    except Exception as e:
        print(f"❌ OpenAI 클라이언트 임베딩 오류: {e}")
        return False

# OpenAI 클라이언트 임베딩 테스트
openai_embedding_success = test_openai_client_embedding()

✅ OpenAI 클라이언트 임베딩 성공:
텍스트: OpenAI 클라이언트로 임베딩을 생성합니다.
임베딩 차원: 1536
임베딩 벡터 (처음 5개): [0.0022130003198981285, -0.021081380546092987, -0.0038472157903015614, -0.02334204502403736, 0.0038574296049773693]

📊 토큰 사용량:
  - 입력 토큰: 19
  - 총 토큰: 19


In [7]:
# Cell 7: 한국어 텍스트 임베딩 특화 테스트
def test_korean_embedding():
    """
    한국어 텍스트에 대한 임베딩 테스트
    """
    if embeddings is None:
        print("❌ 임베딩 모델이 초기화되지 않았습니다.")
        return False
    
    korean_texts = [
        "서울은 대한민국의 수도입니다.",
        "부산은 대한민국 제2의 도시입니다.",
        "김치는 한국의 전통 음식입니다.",
        "한글은 세종대왕이 만든 문자입니다.",
        "K-pop은 전 세계적으로 인기가 있습니다."
    ]
    
    try:
        print("한국어 텍스트 임베딩 생성 중...")
        
        korean_embeddings = embeddings.embed_documents(korean_texts)
        
        print(f"\n✅ 한국어 임베딩 성공:")
        print(f"처리된 한국어 텍스트 수: {len(korean_embeddings)}")
        
        # 도시 관련 텍스트들의 유사도 확인
        seoul_embedding = np.array(korean_embeddings[0])  # 서울
        busan_embedding = np.array(korean_embeddings[1])  # 부산
        kimchi_embedding = np.array(korean_embeddings[2])  # 김치
        
        city_similarity = np.dot(seoul_embedding, busan_embedding) / (
            np.linalg.norm(seoul_embedding) * np.linalg.norm(busan_embedding)
        )
        
        food_similarity = np.dot(seoul_embedding, kimchi_embedding) / (
            np.linalg.norm(seoul_embedding) * np.linalg.norm(kimchi_embedding)
        )
        
        print(f"\n📊 한국어 의미 분석:")
        print(f"서울-부산 유사도 (도시 관련): {city_similarity:.4f}")
        print(f"서울-김치 유사도 (다른 주제): {food_similarity:.4f}")
        
        if city_similarity > food_similarity:
            print("✅ 의미적으로 관련된 텍스트들이 더 높은 유사도를 보입니다.")
        
        return True
        
    except Exception as e:
        print(f"❌ 한국어 임베딩 오류: {e}")
        return False

# 한국어 임베딩 테스트
korean_success = test_korean_embedding()

한국어 텍스트 임베딩 생성 중...

✅ 한국어 임베딩 성공:
처리된 한국어 텍스트 수: 5

📊 한국어 의미 분석:
서울-부산 유사도 (도시 관련): 0.8913
서울-김치 유사도 (다른 주제): 0.8161
✅ 의미적으로 관련된 텍스트들이 더 높은 유사도를 보입니다.


In [8]:
# Cell 8: 대량 텍스트 임베딩 및 성능 테스트
def test_batch_embedding():
    """
    대량 텍스트 임베딩 및 성능 테스트
    """
    if embeddings is None:
        print("❌ 임베딩 모델이 초기화되지 않았습니다.")
        return False
    
    # 테스트용 대량 텍스트 생성
    large_texts = [
        f"이것은 테스트 문서 {i}번입니다. 임베딩 성능을 확인하기 위한 샘플 텍스트입니다."
        for i in range(1, 21)  # 20개 텍스트
    ]
    
    try:
        import time
        
        print(f"대량 텍스트 임베딩 테스트 ({len(large_texts)}개 문서)...")
        
        start_time = time.time()
        batch_embeddings = embeddings.embed_documents(large_texts)
        end_time = time.time()
        
        processing_time = end_time - start_time
        
        print(f"\n✅ 대량 임베딩 성공:")
        print(f"처리된 문서 수: {len(batch_embeddings)}")
        print(f"처리 시간: {processing_time:.2f}초")
        print(f"문서당 평균 처리 시간: {processing_time/len(large_texts):.3f}초")
        print(f"임베딩 차원: {len(batch_embeddings[0])}")
        
        return True
        
    except Exception as e:
        print(f"❌ 대량 임베딩 오류: {e}")
        return False

# 대량 임베딩 테스트
batch_success = test_batch_embedding()

대량 텍스트 임베딩 테스트 (20개 문서)...

✅ 대량 임베딩 성공:
처리된 문서 수: 20
처리 시간: 1.50초
문서당 평균 처리 시간: 0.075초
임베딩 차원: 1536


In [9]:
# Cell 9: 전체 임베딩 테스트 결과 요약
print("\n" + "="*60)
print("=== 전체 임베딩 테스트 결과 요약 ===")

results = {
    "기본 임베딩 테스트": embedding_success if 'embedding_success' in locals() else False,
    "유사도 계산 테스트": similarity_success if 'similarity_success' in locals() else False,
    "OpenAI 클라이언트 테스트": openai_embedding_success if 'openai_embedding_success' in locals() else False,
    "한국어 임베딩 테스트": korean_success if 'korean_success' in locals() else False,
    "대량 임베딩 테스트": batch_success if 'batch_success' in locals() else False
}

for test_name, success in results.items():
    status = "✅ 성공" if success else "❌ 실패"
    print(f"{test_name}: {status}")

success_count = sum(results.values())
total_tests = len(results)

if success_count == total_tests:
    print(f"\n🎉 모든 테스트가 성공했습니다! ({success_count}/{total_tests})")
    print("text-embedding-ada-002 모델이 정상적으로 작동합니다.")
elif success_count > 0:
    print(f"\n⚠️ 일부 테스트가 성공했습니다. ({success_count}/{total_tests})")
    print("부분적으로 임베딩 기능을 사용할 수 있습니다.")
else:
    print(f"\n❌ 모든 테스트가 실패했습니다.")
    print("임베딩 설정이나 API 키를 확인해주세요.")

print(f"\n📋 사용 가능한 기능:")
print(f"- 단일/다중 텍스트 임베딩 생성")
print(f"- 텍스트 간 유사도 계산")
print(f"- 한국어 텍스트 처리")
print(f"- 대량 텍스트 배치 처리")
print(f"- 토큰 사용량 추적")


=== 전체 임베딩 테스트 결과 요약 ===
기본 임베딩 테스트: ❌ 실패
유사도 계산 테스트: ❌ 실패
OpenAI 클라이언트 테스트: ✅ 성공
한국어 임베딩 테스트: ✅ 성공
대량 임베딩 테스트: ✅ 성공

⚠️ 일부 테스트가 성공했습니다. (3/5)
부분적으로 임베딩 기능을 사용할 수 있습니다.

📋 사용 가능한 기능:
- 단일/다중 텍스트 임베딩 생성
- 텍스트 간 유사도 계산
- 한국어 텍스트 처리
- 대량 텍스트 배치 처리
- 토큰 사용량 추적
